<a href="https://colab.research.google.com/github/Miko-Knight/SD_webui_colab/blob/main/SD_webui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Diffusion WebUI Colab

This notebook is made for simple 1 click launch and safe tunneling of Stable Diffusion WebUI. It connects to your Google drive for easy model and output management without much hassle. No or as few as possible third party scripts are used to increase security.

In [ ]:
#@title <b>1. Start the host and check the GPU you were assigned</b>
!nvidia-smi

In [ ]:
#@title <b>2. Connect to your Google Drive or use local directory</b>
import os

#@markdown **Default variables**
models_path = "/content/models" #@param {type:"string"}
vae_path = "/content/vae" #@param {type:"string"}
ti_path = "/content/ti" #@param {type:"string"}
output_path = "/content/output" #@param {type:"string"}

#@markdown **Google Drive variables**
mount_google_drive = True #@param {type:"boolean"}
force_remount = False
if mount_google_drive:
    from google.colab import drive
    try:
        drive_path = "/content/drive"
        drive.mount(drive_path,force_remount=force_remount)
        models_path_gdrive = "/content/drive/MyDrive/AI/StableDiffusion/models" #@param {type:"string"}
        vae_path_gdrive = "/content/drive/MyDrive/AI/StableDiffusion/vae" #@param {type:"string"}
        ti_path_gdrive = "/content/drive/MyDrive/AI/StableDiffusion/ti" #@param {type:"string"}
        output_path_gdrive = "/content/drive/MyDrive/AI/StableDiffusion/output" #@param {type:"string"}
        models_path = models_path_gdrive
        output_path = output_path_gdrive
        vae_path = vae_path_gdrive
        ti_path = ti_path_gdrive
        mount_success = True
    except:
        print("...error mounting drive or with drive path variables")
        print("...reverting to default path variables")
        mount_success = False

os.makedirs(models_path, exist_ok=True)
os.makedirs(vae_path, exist_ok=True)
os.makedirs(ti_path, exist_ok=True)
os.makedirs(output_path, exist_ok=True)

print(f"models_path: {models_path}")
print(f"vae_path: {vae_path}")
print(f"ti_path: {ti_path}")
print(f"output_path: {output_path}")

In [ ]:
#@title <b>3. Install general dependencies</b>
import os
from google.colab.output import eval_js

%cd /content/
os.environ['colab_url'] = eval_js("google.colab.kernel.proxyPort(7860, {'cache': false})")

#General updates and dependencies
!apt -y update -qq
!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install -qq libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb

!pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu116 -U
!apt -y install -qq aria2
!pip install -q --pre xformers==0.0.17.dev476 -U
!pip install -q --pre triton

In [ ]:
#@title <b>(OPTIONAL) Pick any models you want to use and launch the script to download them </b>
import os

#@markdown **Choose your Checkpoint model**
model = "Pastel Mix" #@param ["None", "Anything v3.0", "Grapefruit v4.1","AOM3","Pastel Mix"] {allow-input: true}
vae = "kl-f8-anime2" #@param ["None","pastel-waifu-diffusion","kl-f8-anime2","animefull-final (BlueMoonMix)","vae-ft-mse-840000-ema (Good with DisillusionMix)","orangemix"] {allow-input: true}
ti = "None" #@param ["None","EasyNegative"] {allow-input: true}

#This is where all the models and their links are located
if model == "None":
  model_download_url = ""
  model_file_name = ""
elif model == "Anything v3.0":
  model_download_url = "https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-fp16-pruned.safetensors"
  model_file_name = "anything-v3-fp16-pruned.safetensors"
elif model == "Grapefruit v4.1":
  model_download_url = "https://civitai.com/api/download/models/16619"
  model_file_name = "grapefruitHentaiModel_grapefruitv41.safetensors"
elif model == "Pastel Mix":
  model_download_url = "https://huggingface.co/andite/pastel-mix/resolve/main/pastelmix-fp16.safetensors"
  model_file_name = "pastelmix-fp16.safetensors"
elif model == "Andromeda-Mix":
  model_download_url = "https://civitai.com/api/download/models/7671"
  model_file_name = "andromedaMix_andromedaVersion2.safetensors"
elif model == "Cetus-Mix Coda Edition":
  model_download_url = "https://civitai.com/api/download/models/20229"
  model_file_name = "cetusMix_Codaedition.safetensors"
elif model == "Cetus-Mix V3":
  model_download_url = "https://civitai.com/api/download/models/18496"
  model_file_name = "cetusMix_cetusVersion3.safetensors"
elif model == "ColorBomb":
  model_download_url = "https://huggingface.co/mocker/KaBoom/resolve/main/ColorBombMix-fp16-no-ema.safetensors"
  model_file_name = "ColorBombMix-fp16-no-ema.safetensors"
elif model == "FaceBomb":
  model_download_url = "https://huggingface.co/mocker/KaBoom/resolve/main/FaceBombMix-fp16-no-ema.safetensors"
  model_file_name = "FaceBombMix-fp16-no-ema.safetensors"
elif model == "HyperBomb":
  model_download_url = "https://huggingface.co/mocker/KaBoom/resolve/main/HyperBombMix-fp16-no-ema.safetensors"
  model_file_name = "HyperBombMix-fp16-no-ema.safetensors"
elif model == "VividibiMix":
  model_download_url = "https://civitai.com/api/download/models/9644"
  model_file_name = "vividibimix_.safetensors"
elif model == "WhiteSpace Prism":
  model_download_url = "https://civitai.com/api/download/models/15240?type=Pruned%20Model&format=SafeTensor"
  model_file_name = "whitespace_Prism.safetensors"
elif model == "BlueMoonMix":
  model_download_url = "https://civitai.com/api/download/models/23726"
  model_file_name = "bluemoonmix_v1.safetensors"
elif model == "PileDream":
  model_download_url = "https://civitai.com/api/download/models/26119"
  model_file_name = "piledream_.safetensors"
elif model == "Pretty 2D":
  model_download_url = "https://civitai.com/api/download/models/18377"
  model_file_name = "pretty2D_pretty2DPrunedFp16.safetensors"
elif model == "Pretty 2.5D":
  model_download_url = "https://civitai.com/api/download/models/18371"
  model_file_name = "pretty25D_pretty25DPrunedFp16.safetensors"
elif model == "Pretty 2.5D V2":
  model_download_url = "https://civitai.com/api/download/models/20775"
  model_file_name = "pretty25DV2_pretty25DV2Pruned.safetensors"
elif model == "Kawaii 2D":
  model_download_url = "https://civitai.com/api/download/models/22503"
  model_file_name = "kawaii2D_kawaii2DV11Pruend.safetensors"
elif model == "Kawaii 2.5D":
  model_download_url = "https://civitai.com/api/download/models/22509"
  model_file_name = "kawaii25D_kawaii25DV2Pruend.safetensors"
elif model == "Animelike 2D":
  model_download_url = "https://civitai.com/api/download/models/25853"
  model_file_name = "animelike2D_animelike2DPruned.safetensors"
elif model == "Animelike 2.5D":
  model_download_url = "https://civitai.com/api/download/models/26176"
  model_file_name = "animelike25D_animelike25DV11Pruned.safetensors"
elif model == "Beauty 2.5D":
  model_download_url = "https://civitai.com/api/download/models/21564"
  model_file_name = "beauty25D_beauty25DPrunedFp16.safetensors"
elif model == "CreamMix":
  model_download_url = "https://civitai.com/api/download/models/23650"
  model_file_name = "creammix_v10.safetensors"
elif model == "Counterfeit v2.5":
  model_download_url = "https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/Counterfeit-V2.5_fp16.safetensors"
  model_file_name = "Counterfeit-V2.5_fp16.safetensors"
elif model == "Counterfeit v2.2":
  model_download_url = "https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/Counterfeit-V2.2.safetensors"
  model_file_name = "Counterfeit-V2.2.safetensors"
elif model == "ChilloutMix":
  model_download_url = "https://civitai.com/api/download/models/11732"
  model_file_name = "chilloutmix_NiPrunedFp16Fix.safetensors"
elif model == "refslaveV2_v2 (good for environmental stuff)":
  model_download_url = "https://huggingface.co/Dorshu/refslaveV2_v2/resolve/main/refslaveV2_v2.safetensors"
  model_file_name = "refslaveV2_v2.safetensors"
elif model == "WinterMoonMix A":
  model_download_url = "https://civitai.com/api/download/models/15708"
  model_file_name = "wintermoonmix_A.safetensors"
elif model == "CyriousMix":
  model_download_url = "https://civitai.com/api/download/models/7951"
  model_file_name = "cyriousmix_14.safetensors"
elif model == "KuromiMix V1.0":
  model_download_url = "https://civitai.com/api/download/models/25529"
  model_file_name = "kuromimix_V10.safetensors"
elif model == "KuromiMix V2.0":
  model_download_url = "https://civitai.com/api/download/models/25578"
  model_file_name = "kuromimix_V20.ckpt"
elif model == "MeinaUnreal":
  model_download_url = "https://civitai.com/api/download/models/22314"
  model_file_name = "meinaunreal_v1Beta.safetensors"
elif model == "CamelliaBlossomMix":
  model_download_url = "https://civitai.com/api/download/models/21759"
  model_file_name = "camelliablossommix_finaledition.safetensors"
elif model == "EtherRealMix v1.1":
  model_download_url = "https://civitai.com/api/download/models/23036"
  model_file_name = "etherRealMix_V11.safetensors"
elif model == "DisillusionMix v3":
  model_download_url = "https://civitai.com/api/download/models/21229"
  model_file_name = "disillusionmix_3.safetensors"
elif model == "DisillusionMix v2":
  model_download_url = "https://civitai.com/api/download/models/20544"
  model_file_name = "disillusionmix_2.safetensors"
elif model == "AOM3":
  model_download_url = "https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3_orangemixs.safetensors"
  model_file_name = "AOM3_orangemixs.safetensors"
elif model == "AOM3A1":
  model_download_url = "https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A1_orangemixs.safetensors"
  model_file_name = "AOM3A1_orangemixs.safetensors"
elif model == "AOM3A1B":
  model_download_url = "https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A1B_orangemixs.safetensors"
  model_file_name = "AOM3A1B_orangemixs.safetensors"
elif model == "AOM3A2":
  model_download_url = "https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A2_orangemixs.safetensors"
  model_file_name = "AOM3A2_orangemixs.safetensors"
elif model == "AOM3A3":
  model_download_url = "https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A3_orangemixs.safetensors"
  model_file_name = "AOM3A3_orangemixs.safetensors"
elif model == "Fish-mix":
  model_download_url = "https://civitai.com/api/download/models/18583"
  model_file_name = "MixFishMix_v10.safetensors"
elif model == "Mechanicmix_V2 (2D)":
  model_download_url = "https://civitai.com/api/download/models/18955"
  model_file_name = "mechanicmixV2_mechanicmixV2.safetensors"
elif model == "Mechanicmix_V1 (2.5D)":
  model_download_url = "https://civitai.com/api/download/models/17532"
  model_file_name = "mechanicmixV2_mechanicmixV1.safetensors"
elif model == "":
  model_download_url = ""
  model_file_name = ""

#This is where all the vae and their links are located
if vae == "None":
  vae_download_url = ""
  vae_file_name = ""
elif vae == "pastel-waifu-diffusion":
  vae_download_url = "https://huggingface.co/andite/pastel-mix/resolve/main/pastel-waifu-diffusion.vae.pt"
  vae_file_name = "pastel-waifu-diffusion.vae.pt"
elif vae == "kl-f8-anime2":
  vae_download_url = "https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt"
  vae_file_name = "kl-f8-anime2.ckpt"
elif vae == "animefull-final (BlueMoonMix)":
  vae_download_url = "https://civitai.com/api/download/models/23726?type=VAE&format=Other"
  vae_file_name = "animefull-final-pruned.vae.pt"
elif vae == "vae-ft-mse-840000-ema (Good with DisillusionMix)":
  vae_download_url = "https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors"
  vae_file_name = "vae-ft-mse-840000-ema-pruned.safetensors"
elif vae == "orangemix":
  vae_download_url = "https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt"
  vae_file_name = "orangemix.vae.pt"
elif vae == "":
  vae_download_url = ""
  vae_file_name = ""

#This is where Textual Inversions and their links are located
if ti == "None":
  ti_download_url = ""
  ti_file_name = ""
elif ti == "EasyNegative":
  ti_download_url = "https://huggingface.co/datasets/gsdf/EasyNegative/resolve/main/EasyNegative.safetensors"
  ti_file_name = "EasyNegative.safetensors"


#Download everything
if model != "None":
  model_download = "aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {} -d {} -o {}".format(model_download_url, models_path, model_file_name)
else:
  print("No model selected for download. Skipping...")

if vae != "None":
  vae_download = "aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {} -d {} -o {}".format(vae_download_url, vae_path, vae_file_name)
else:
  print("No vae selected for download. Skipping...")

if ti !="None":
  ti_download = "aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {} -d {} -o {}".format(ti_download_url, ti_path, ti_file_name)
else:
  print("No textual inversion selected for download. Skipping...")

!{model_download}
!{vae_download}
!{ti_download}

In [ ]:
#@title <b>4. Download dependencies and Stable Diffusion WebUI (AUTOMATIC1111)</b>

# This whole part needs to be rechecked and made sure it's alright besides the last line
import os
from google.colab.output import eval_js

#@markdown **Choose your tunnel provider (doesn't work right now - always uses cloudflared)**
provider = "cloudflared" #@param ["Localtunnel", "cloudflared"]

%cd /content/
os.environ['colab_url'] = eval_js("google.colab.kernel.proxyPort(7860, {'cache': false})")

!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui #this is the only 100% legit one
!git clone https://github.com/deforum-art/deforum-for-automatic1111-webui /content/stable-diffusion-webui/extensions/deforum-for-automatic1111-webui
!git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser /content/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser
!git clone https://github.com/kohya-ss/sd-webui-additional-networks /content/stable-diffusion-webui/extensions/sd-webui-additional-networks
!git clone https://github.com/Mikubill/sd-webui-controlnet /content/stable-diffusion-webui/extensions/sd-webui-controlnet
!git clone https://github.com/camenduru/openpose-editor /content/stable-diffusion-webui/extensions/openpose-editor
!git clone https://github.com/jexom/sd-webui-depth-lib /content/stable-diffusion-webui/extensions/sd-webui-depth-lib
!git clone https://github.com/hnmr293/posex /content/stable-diffusion-webui/extensions/posex
!git clone https://github.com/camenduru/sd-webui-tunnels /content/stable-diffusion-webui/extensions/sd-webui-tunnels
!git clone https://github.com/etherealxx/batchlinks-webui /content/stable-diffusion-webui/extensions/batchlinks-webui
%cd /content/stable-diffusion-webui
!git reset --hard

#Download ControlNet extensions
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_canny-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_canny-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_depth-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_depth-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_hed-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_hed-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_mlsd-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_mlsd-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_normal-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_normal-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_openpose-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_openpose-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_scribble-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_scribble-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_seg-fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_seg-fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/hand_pose_model.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/openpose -o hand_pose_model.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/body_pose_model.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/openpose -o body_pose_model.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/dpt_hybrid-midas-501f0c75.pt -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/midas -o dpt_hybrid-midas-501f0c75.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/mlsd_large_512_fp32.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/mlsd -o mlsd_large_512_fp32.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/mlsd_tiny_512_fp32.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/mlsd -o mlsd_tiny_512_fp32.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/network-bsds500.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/hed -o network-bsds500.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/upernet_global_small.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/uniformer -o upernet_global_small.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_style_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_style_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_sketch_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_sketch_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_seg_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_seg_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_openpose_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_openpose_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_keypose_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_keypose_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_depth_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_depth_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_color_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_color_sd14v1.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_canny_sd14v1.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_canny_sd14v1.pth

!sed -i -e '''/    prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/stable-diffusion-webui/launch.py
!sed -i -e 's/fastapi==0.90.1/fastapi==0.89.1/g' /content/stable-diffusion-webui/requirements_versions.txt

!mkdir /content/stable-diffusion-webui/extensions/deforum-for-automatic1111-webui/models

In [ ]:
#@title <b>5. Start Stable Diffusion WebUi + Tunnel</b>
import os
!python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --multiple --ckpt-dir $models_path --vae-dir $vae_path --embeddings-dir $ti_path